# Strategy Analysis Example

Run a backtest on the single indicator template and plot the results.

In [ ]:
import inspect
import pandas as pd
import matplotlib.pyplot as plt

from qmtl.examples.templates.single_indicator import SingleIndicatorStrategy
from qmtl.examples.defaults import load_backtest_defaults
from qmtl.sdk import Runner

cfg = load_backtest_defaults(inspect.getfile(SingleIndicatorStrategy))
strategy = Runner.backtest(
    SingleIndicatorStrategy,
    start_time=cfg.get('start_time'),
    end_time=cfg.get('end_time'),
    on_missing=cfg.get('on_missing', 'skip'),
    gateway_url='http://localhost:8000'
)

price_snap = strategy.nodes[0].cache._snapshot()[strategy.nodes[0].node_id][strategy.nodes[0].interval]
ema_snap = strategy.nodes[1].cache._snapshot()[strategy.nodes[1].node_id][strategy.nodes[1].interval]
price = pd.DataFrame(price_snap, columns=['ts', 'payload'])
price['close'] = price['payload'].apply(lambda p: p['close'])
ema = pd.DataFrame(ema_snap, columns=['ts', 'ema'])
merged = pd.merge(price[['ts', 'close']], ema, on='ts')
merged.set_index('ts')[['close', 'ema']].plot()
plt.show()
